In [ ]:
test_set_path = "../../../datasets/test_set.csv"

In [ ]:
import pandas as pd

test_df = pd.read_csv(test_set_path)

print("Test Set Used:", test_df.shape)

In [ ]:
X_test = test_df.drop(
    columns=["attack_binary", "attack_categorical", "attack_class"]
).values

y_test = test_df["attack_binary"].values

y_test_class = test_df["attack_class"].values

print(f"test set count: {test_df.shape[0]:,} with {test_df.shape[1]:,} features")
print(f"unique values: {test_df['attack_class'].value_counts()}")
test_df.head(3)

In [ ]:
import onnxruntime as ort

ocsvm_onnx_path = "saved_models/onnx/base_ocsvm_nsl_kdd.onnx"

# Create an ONNX Runtime session
session = ort.InferenceSession(ocsvm_onnx_path)

# Prepare some test data
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

# Run inference
test_pred_onnx = session.run(
    [output_name], {input_name: X_test.astype("float32")}
)[0]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, test_pred_onnx, labels=[-1, 1])


def plot_confusion_matrix(cm, labels, title):
    plt.figure(figsize=(5, 4))
    sns.heatmap(
        cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels
    )
    plt.xlabel("Predicted Label")
    plt.ylabel("Actual Label")
    plt.title(title)
    plt.show()


print("Confusion Matrix of ONNX OCSVM Predictions")
plot_confusion_matrix(cm, ["Anomaly", "Normal"], "Confusion Matrix (Anomaly vs Normal)")

In [ ]:
from sklearn.metrics import (
    classification_report,
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
)

print("Classification Report ONNX:")
print(classification_report(y_test, test_pred_onnx, target_names=["Anomaly", "Normal"]))

precision = precision_score(y_test, test_pred_onnx, pos_label=-1)
recall = recall_score(y_test, test_pred_onnx, pos_label=-1)
f1 = f1_score(y_test, test_pred_onnx, pos_label=-1)

print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")
print(f"Accuracy: {accuracy_score(y_test, test_pred_onnx) * 100:.2f}%")